In [7]:
import numpy as np
from pydrake.all import (
    ConstantVectorSource,
    DiagramBuilder,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    MultibodyPlant,
    Parser,
    PiecewisePolynomial,
    PiecewiseQuaternionSlerp,
    RigidTransform,
    RollPitchYaw,
    RotationMatrix,
    Simulator,
    Solve,
    StartMeshcat,
    TrajectorySource,
)

In [5]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7002


In [ ]:
builder = DiagramBuilder()

plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0)
parser = Parser(plant)
ConfigureParser(parser)
parser.AddModelsFromUrl("package://manipulation/iiwa_and_wsg.dmd.yaml")
plant.Finalize()

In [8]:
import numpy as np
from IPython.display import clear_output, display
from pydrake.all import (
    AbstractValue,
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    JointSliders,
    LeafSystem,
    MeshcatVisualizer,
    Parser,
    RigidTransform,
    RollPitchYaw,
    StartMeshcat,
)

from manipulation import ConfigureParser, running_as_notebook
from manipulation.scenarios import AddMultibodyTriad

In [9]:
class PrintPose(LeafSystem):
    def __init__(self, body_index):
        LeafSystem.__init__(self)
        self._body_index = body_index
        self.DeclareAbstractInputPort(
            "body_poses", AbstractValue.Make([RigidTransform()])
        )
        self.DeclareForcedPublishEvent(self.Publish)

    def Publish(self, context):
        pose = self.get_input_port().Eval(context)[self._body_index]
        print(pose)
        print(
            "gripper position (m): "
            + np.array2string(
                pose.translation(),
                formatter={"float": lambda x: "{:3.2f}".format(x)},
            )
        )
        print(
            "gripper roll-pitch-yaw (rad):"
            + np.array2string(
                RollPitchYaw(pose.rotation()).vector(),
                formatter={"float": lambda x: "{:3.2f}".format(x)},
            )
        )
        clear_output(wait=True)


def gripper_forward_kinematics_example():
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0)
    parser = Parser(plant)
    ConfigureParser(parser)
    parser.AddModelsFromUrl("package://manipulation/iiwa_and_wsg.dmd.yaml")
    plant.Finalize()

    # Draw the frames
    for body_name in [
        "iiwa_link_1",
        "iiwa_link_2",
        "iiwa_link_3",
        "iiwa_link_4",
        "iiwa_link_5",
        "iiwa_link_6",
        "iiwa_link_7",
        "body",
    ]:
        pass
        # AddMultibodyTriad(plant.GetFrameByName(body_name), scene_graph)

    meshcat.Delete()
    meshcat.DeleteAddedControls()
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph.get_query_output_port(), meshcat
    )

    wsg = plant.GetModelInstanceByName("wsg")
    gripper = plant.GetBodyByName("body", wsg)
    print_pose = builder.AddSystem(PrintPose(gripper.index()))
    builder.Connect(
        plant.get_body_poses_output_port(), print_pose.get_input_port()
    )

    default_interactive_timeout = None if running_as_notebook else 1.0
    sliders = builder.AddSystem(JointSliders(meshcat, plant))
    diagram = builder.Build()
    sliders.Run(diagram, default_interactive_timeout)
    meshcat.DeleteAddedControls()


gripper_forward_kinematics_example()

INFO:drake:PackageMap: Downloading https://github.com/RobotLocomotion/models/archive/fe5326c5ffc36fda12c58883d22d29dc86009d65.tar.gz
sh: -c: line 0: syntax error near unexpected token `('
sh: -c: line 0: `/usr/bin/python3 /Users/timkostolansky/Dropbox (MIT)/MIT Semesters/Fall23/6.4212/project/env/lib/python3.11/site-packages/pydrake/lib/../share/drake/multibody/parsing/package_downloader.py /private/var/tmp/.cache_timkostolansky/drake/package_map/.fetch_zmY3Ws /private/var/tmp/.cache_timkostolansky/drake/package_map/.error_OKpDIQ --disable-drake-valgrind-tracing'


RuntimeError: PackageMap: when downloading 'drake_models', the downloader experienced an error: returncode == 512